In [1]:
import numpy as np
import copy
import math
from numba import jit
from numba import prange

import cProfile, pstats, io
from pstats import SortKey

In [2]:
side_len = 5
max_listed = 15
stones = sum([n+1 for n in range(side_len)])
Shortest = stones

In [3]:
def find_unique_starts(side_length):
    starts = []
    # nested triangles as layers, the top half (rounded up) of the side is unique
    layer_peak = 0
    for layer in range(math.ceil(side_length/3)):
        layer_peak += layer * 4
        layer_side = side_length - layer * 3
        for row in range(math.ceil(layer_side/2)):
            starts.append(layer_peak + row * (layer * 2)
                                 + sum([x for x in range(row + 1)]))
    return starts

In [4]:
def add_move(start, middle, end, m_list, chk_list, perf_list):
    # adds the jump and its reverse to the lists provided
    m_list.append((start, end))
    m_list.append((end, start))
    arr = np.zeros(stones)
    arr[start] = .3
    arr[middle] = .3
    arr[end] = -.3
    chk_list.append(copy.copy(arr))
    arr[end] = .3
    arr[start] = -.3
    chk_list.append(copy.copy(arr))
    arr[start] = -1
    arr[middle] = -1
    arr[end] = 1
    perf_list.append(copy.copy(arr))
    arr[start] = 1
    arr[end] = -1
    perf_list.append(copy.copy(arr))
    
def get_moves(side_length):    
    possible_moves = []
    check_moves_list = []
    perform_moves_list = []
    start_of_row = 0
    # row of move's start
    for n in range(side_length):
        start_of_row += n
        # column of move's start
        for m in range(n+1):
            start = start_of_row + m
            # 'down' and 'angled'
            if side_length >= 1 + (n + 2):
                add_move(start, start + n + 1, start + 2*n + 3, possible_moves, check_moves_list, perform_moves_list)
                add_move(start, start + n + 2, start + 2*n + 5, possible_moves, check_moves_list, perform_moves_list)

            # horizontal
            if n >= m + 2:
                add_move(start, start + 1, start + 2, possible_moves, check_moves_list, perform_moves_list)
                
    return possible_moves, np.array(check_moves_list), np.array(perform_moves_list, dtype = int)


In [5]:
print("Analyzing Board:")
if side_len <= 4:
    spacing = 1
elif side_len < 14:
    spacing = 2
else:
    spacing = 3

row_start = 0
for row in range(side_len):
    row_start += row        
    for colu in range(row + 1):
        if row_start > 0:
            print(row_start + colu, end = " " * (spacing - int(math.log10(row_start + colu))))
        else:
            print(0, end = "")
    print()

Analyzing Board:
0
1  2  
3  4  5  
6  7  8  9  
10 11 12 13 14 


In [6]:
moves, check, perform = get_moves(side_len)
unique_starts = find_unique_starts(side_len)
print("Unique Starting Positions:")
print(unique_starts)
print()

print("All Potentially Valid Moves:")
print(moves)

Unique Starting Positions:
[0, 1, 3, 4]

All Potentially Valid Moves:
[(0, 3), (3, 0), (0, 5), (5, 0), (1, 6), (6, 1), (1, 8), (8, 1), (2, 7), (7, 2), (2, 9), (9, 2), (3, 10), (10, 3), (3, 12), (12, 3), (3, 5), (5, 3), (4, 11), (11, 4), (4, 13), (13, 4), (5, 12), (12, 5), (5, 14), (14, 5), (6, 8), (8, 6), (7, 9), (9, 7), (10, 12), (12, 10), (11, 13), (13, 11), (12, 14), (14, 12)]


In [7]:
def record_success(previous_moves, valid_moves, success_list, max_listed):
    concat_moves = [[previous_moves[0]]]
    for move in previous_moves[1:]:
        if valid_moves[move][0] == concat_moves[-1][-1]:
            concat_moves[-1].append(valid_moves[move][1])
        else:
            concat_moves.append([valid_moves[move][0], valid_moves[move][1]])

    if max_listed:
        global Shortest
        if len(concat_moves) < Shortest:
            print("New shortest: ", end = "")
            print(concat_moves)
            success_list.clear()
            success_list.append(concat_moves)
            Shortest = len(concat_moves)
        elif len(concat_moves) == Shortest and len(success_list) < max_listed:
            success_list.append(concat_moves)
            print("Equal length: ", end = "")
            print(concat_moves)
        #elif len(concat_moves) == Shortest:
            #print("Equal length: ", end = "")
            #print(concat_moves)

    else:
        success_list.append(concat_moves)
        
    return 0

@jit(parallel = True)
def get_move_array(game_state, check_array, perform_array):
    #print(game_state)
    #print(check_array)
    #print(game_state.dot(check_array.T))
    #print((game_state.dot(check_array.T) + .5)//1)
    return ((game_state.dot(check_array.T) + .5)//1 * perform_array.T).T

#@jit(parallel = True)
def play_game(game_state, check_array, perform_array, success_list, previous_moves, valid_moves, max_listed = None):
    # First term of previous moves is the starting removed stone, later terms refer to the index of a valid move
    #move_array = np.zeros((len(valid_moves), (len(game_state))))
    move_array = get_move_array(game_state.astype(float), check_array, perform_array)
    n = move_array.shape[0]
    #move_array = ((game_state.dot(check_array.T) + .5)//1 * perform_array.T).T
    for i in range(n):
        if np.any(move_array[i]):
            #print(move_array[i])
            new_moves = copy.copy(previous_moves)
            new_moves.append(i)
            play_game(game_state + move_array[i], check_array, perform_array, success_list, new_moves, valid_moves, max_listed)
        
    if np.sum(game_state) == 1:
        record_success(previous_moves, valid_moves, success_list, max_listed)



In [8]:
complete_games = []
Shortest = 11

pr = cProfile.Profile()
pr.enable()

for start in unique_starts:
    starting_stones = np.ones(stones, dtype = int) 
    starting_stones[start] = 0
    play_game(starting_stones, check, perform, complete_games, [start], moves, 15)
    
pr.disable()
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

Equal length: [[0], [3, 0], [10, 3], [8, 6, 1], [2, 7], [11, 4], [9, 2, 7], [13, 11], [0, 3, 12], [11, 13], [14, 12]]
Equal length: [[0], [3, 0], [10, 3], [8, 6, 1], [2, 7], [11, 4], [13, 11], [9, 2, 7], [0, 3, 12], [11, 13], [14, 12]]
Equal length: [[0], [3, 0], [12, 3], [6, 1], [2, 7], [9, 2], [10, 12], [13, 11], [0, 3, 12], [11, 13], [14, 12, 5, 0]]
Equal length: [[0], [3, 0], [12, 3], [6, 1], [2, 7], [9, 2], [10, 12], [13, 11], [0, 5, 12], [11, 13], [14, 12, 3, 0]]
Equal length: [[0], [3, 0], [12, 3], [6, 1], [2, 7], [9, 2], [10, 12], [13, 11, 4], [0, 3, 5], [2, 9], [14, 5, 12]]
Equal length: [[0], [3, 0], [12, 3], [6, 1], [2, 7], [9, 2], [14, 12], [11, 13], [0, 3, 12], [13, 11], [10, 12, 5, 0]]
Equal length: [[0], [3, 0], [12, 3], [6, 1], [2, 7], [9, 2], [14, 12], [11, 13], [0, 5, 12], [13, 11], [10, 12, 3, 0]]
Equal length: [[0], [3, 0], [12, 3], [6, 1], [2, 7], [9, 2], [14, 12], [11, 13, 4], [0, 5, 3], [1, 6], [10, 3, 12]]
Equal length: [[0], [3, 0], [12, 3], [6, 1], [2, 7], [10

In [9]:
"""
min_length = len(starting_stones)
for lis in complete_games:
    if len(lis) < min_length:
        min_length = len(lis)
print()
print("Minimum Solution Length: " + str(min_length))
no_printed = 0
for lis in complete_games:
    if no_printed >= max_listed:
        print("...")
        break
    if len(lis) == min_length:
        print(lis)
        no_printed += 1
"""

'\nmin_length = len(starting_stones)\nfor lis in complete_games:\n    if len(lis) < min_length:\n        min_length = len(lis)\nprint()\nprint("Minimum Solution Length: " + str(min_length))\nno_printed = 0\nfor lis in complete_games:\n    if no_printed >= max_listed:\n        print("...")\n        break\n    if len(lis) == min_length:\n        print(lis)\n        no_printed += 1\n'

In [10]:
bool_arr = np.array([0,1,1,1,0,0,1,1,1,1], dtype = bool, ndmin = 2)

In [11]:
bool_arr.dot(np.array([[-.5,.5,.5,0,0,0,0,0,0,0],[0,.5,0,0,.5,0,0,-.5,0,0]], dtype = float, ndmin = 2).T).astype(int).astype(bool)

array([[ True, False]])

In [12]:
x = np.array([0,1,2.5,3.5,-.5,-1.5])
x//1

array([ 0.,  1.,  2.,  3., -1., -2.])